In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_diab=pd.read_csv('diabetes.csv')

In [3]:
df_diab.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df_diab.Outcome.unique()

array([1, 0])

In [5]:
df_diab.shape

(768, 9)

In [8]:
df_diab.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [10]:
X=df_diab.drop(['Outcome'],axis=1)
y=df_diab[['Outcome']]

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=12345)

In [13]:
from sklearn.preprocessing import StandardScaler

In [15]:
scaler=StandardScaler()

In [16]:
X_train_scaled=scaler.fit_transform(X_train)

In [17]:
X_test_scaled=scaler.transform(X_test)

In [18]:
import tensorflow as tf


In [19]:
from tensorflow.keras.models import Sequential

In [20]:
from tensorflow.keras.layers import Dense, Dropout

In [21]:
tf_model=Sequential()

In [22]:
tf_model.add(Dense(units=128,activation='relu',input_shape=(X_train_scaled.shape[1],)))

In [24]:
tf_model.add(Dropout(0.2,seed=12345))

In [25]:
tf_model.add(Dense(units=64,activation='relu'))


In [26]:
tf_model.add(Dropout(0.2,seed=12345))

In [27]:
tf_model.add(Dense(units=1,activation='sigmoid'))

In [28]:
tf_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1152      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 9,473
Trainable params: 9,473
Non-trainable params: 0
_________________________________________________________________


In [29]:
tf_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
tf_model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
20/20 [==============================] - 1s 4ms/step - loss: 0.6254 - accuracy: 0.6547
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5344 - accuracy: 0.7427
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4995 - accuracy: 0.7687
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4814 - accuracy: 0.7687
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4727 - accuracy: 0.7736
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4699 - accuracy: 0.7769
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4653 - accuracy: 0.7801
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4621 - accuracy: 0.7769
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4516 - accuracy: 0.7801
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4499 - accuracy: 0.7834
Epoch 11/

In [31]:
y_test_pred=tf_model.predict(X_test_scaled)>0.5

In [32]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [34]:
confusion_matrix(y_pred=y_test_pred,y_true=y_test)

array([[88, 18],
       [19, 29]])

In [35]:
accuracy_score(y_pred=y_test_pred,y_true=y_test)

0.7597402597402597

In [36]:
# Hyperparameter Tunning

In [43]:
def build_model(hp):
    model=Sequential()
    model.add(Dense(units=hp.Int("units",min_value=32,max_value=1024,step=16),
                    activation=hp.Choice("activation",["relu","tanh"]),
                    input_shape=(X_train_scaled.shape[1],)))
    model.add(Dropout(hp.Float("rate",min_value=0.1,max_value=0.4,step=0.1),seed=12345))
    
    model.add(Dense(units=hp.Int("units",min_value=32,max_value=1024,step=16),
                    activation=hp.Choice("activation",["relu","tanh"])))
    model.add(Dropout(hp.Float("rate",min_value=0.1,max_value=0.4,step=0.1),seed=12345))
    
    model.add(Dense(units=1,activation='sigmoid'))
    
    learning_rate=hp.Float("learning_rate",min_value=0.001,max_value=0.1,step=0.01)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                  loss='binary_crossentropy',metrics=['accuracy'])
    return model
                    
            
    

In [44]:
import keras_tuner as kt
build_model(kt.HyperParameters())

In [48]:
rtuner=kt.RandomSearch(hypermodel=build_model,
                       objective="val_accuracy",
                       max_trials=3,
                       executions_per_trial=2,
                       overwrite=True,
                       directory="my_dir",
                       project_name="diab")

In [51]:
rtuner.search(X_train_scaled,y_train,epochs=2,validation_data=(X_test_scaled,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.7272727489471436

Best val_accuracy So Far: 0.8019480407238007
Total elapsed time: 00h 00m 49s
INFO:tensorflow:Oracle triggered exit


In [52]:
models=rtuner.get_best_models(num_models=2)

In [57]:
models[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 224)               2016      
                                                                 
 dropout (Dropout)           (None, 224)               0         
                                                                 
 dense_1 (Dense)             (None, 224)               50400     
                                                                 
 dropout_1 (Dropout)         (None, 224)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 225       
                                                                 
Total params: 52,641
Trainable params: 52,641
Non-trainable params: 0
_________________________________________________________________


In [58]:
y_test_pred=tf_model.predict(X_test_scaled)>0.5

In [59]:
accuracy_score(y_pred=y_test_pred,y_true=y_test)

0.7597402597402597

In [60]:
rtuner.get_best_hyperparameters()[0].get_config()['values']

{'units': 224,
 'activation': 'relu',
 'rate': 0.30000000000000004,
 'learning_rate': 0.001}